<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/games_400_FindTheCar_template/games_400_FindTheCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture 
!pip install pennylane

In [ ]:
from pennylane import numpy as np
import pennylane as qml
dev = qml.device("default.qubit", wires=[0, 1, "sol"], shots=1)

This challenge is similar to the famous [Monty Hall Problem](https://en.wikipedia.org/wiki/Monty_Hall_problem). The difference is that, in this ocassion the player has to guess 1 of 4 doors to find the car. Additinally, instead of being able to check the doors, we can only query an oracle that encodes which of the four doors hides the car.

The doors are represented by the 4 possible 2 qubit basis states $|00\rangle, |01\rangle |10\rangle, |11\rangle$ and the oracle outputs $|1\rangle$ when it is queried with the state corresponding to the door that hides the car and $|0\rangle$ otherwise.

The goal of this challenge is to find the car with 100% certainty only querying the oracle twice and only using single qubit gates.

In [ ]:
def find_the_car(oracle):
    """Function which, given an oracle, returns which door that the car is behind.

    Args:
        - oracle (function): function that will act as an oracle. The first two qubits (0,1)
        will refer to the door and the third ("sol") to the answer.

    Returns:
        - (int): 0, 1, 2, or 3. The door that the car is behind.
    """
    @qml.qnode(dev)
    def circuit1():
        # QHACK #
        qml.Hadamard(wires=0)
        qml.PauliX(wires="sol")
        qml.Hadamard(wires="sol")
        oracle()
        qml.Hadamard(wires="sol")
        qml.PauliX(wires="sol")
        qml.Hadamard(wires=0)
        # QHACK #
        return qml.sample()

    @qml.qnode(dev)
    def circuit2():
        # QHACK #
        qml.Hadamard(wires=1)
        qml.PauliX(wires="sol")
        qml.Hadamard(wires="sol")
        oracle()
        qml.Hadamard(wires="sol")
        qml.PauliX(wires="sol")
        qml.Hadamard(wires=1)
        # QHACK #
        return qml.sample()

    sol1 = circuit1()
    sol2 = circuit2()

    # QHACK #

    bit1 = sol1[0]
    bit2 = sol2[1]

    # QHACK #
    if bit1 == 0 and bit2 == 0:
      return 3
    elif bit1 == 1 and bit2 == 1:
      return 0
    elif bit1 == 1 and bit2 == 0:
      return 2
    else:
      return 1

Testing with 1.in. Notice that because of the way the QHack layed out their solutions the binary representation of the door index is switched.

In [ ]:
numbers = [0, 0]

def oracle():
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, "sol"])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)

mapping = {0:3, 1:2, 2:1, 3:0}

output = find_the_car(oracle)
car_door = mapping[numbers[0]*2 + numbers[1]]
print(f"Car is in door {car_door} and we chose {output}")

First, let's take a look at the possible oracles.

In [ ]:
@qml.qnode(dev)
def visualoracle(numbers):
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, "sol"])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    return qml.sample()
for numbers in [[0,0], [0, 1], [1, 0], [0, 0]]: 
  print(numbers, "car in door number", mapping[numbers[0]*2+numbers[1]])
  drawer = qml.draw(visualoracle)
  print(drawer(numbers))

Let's focus our analysis on door 2

In [ ]:
def binary_list(m, n):
    arr = []
    # Create the 0 representation of length n
    for i in range(0, n):
        arr.append(0)
    i = 0
    # Set the elements of arr to represent m
    while m != 0:
        arr[len(arr) - 1 - i] = int(m) % 2
        m = int(m / 2)
        i += 1
    return arr

def get_basis_states(n):
    arr = []
    # Create all possible binary lists from 0 to 2**n
    for i in range(0, 2**n):
        arr.append(binary_list(i, n))
    return arr

def get_state_amplitudes(state):
    i = 0
    m = len(state)
    while m > 1:
      m = m //2
      i += 1
    new_shape = [2]*i
    basis_states = get_basis_states(i)
    state = state.reshape(new_shape)
    for bs in basis_states:
      bs_a = state[bs[0]]
      for i in bs[1:]:
        bs_a = bs_a[i]
      bs_a = np.round(bs_a.real, 3)
      if bs_a != 0:
        st = "|"
        for i in bs:
          st += str(i)
        st += ">"
        print(f"{st}: {bs_a}")
    return st

#get_state_amplitudes(np.array([1]*8))

In [ ]:
numbers = [0, 1]

dev2 = qml.device("default.qubit", wires=[0, 1, 2])

def partial_circuit(i):
    if i > 0:
      qml.Hadamard(wires=0)
      qml.PauliX(wires=2)
      qml.Hadamard(wires=2)
    if i > 1:
      if numbers[0] == 1:
        qml.PauliX(wires=0)
      if numbers[1] == 1:
          qml.PauliX(wires=1)
    if i > 2:
      qml.Toffoli(wires=[0, 1, 2])
    if i > 3:
      if numbers[0] == 1:
        qml.PauliX(wires=0)
      if numbers[1] == 1:
          qml.PauliX(wires=1)
    if i > 4:
      qml.Hadamard(wires=2)
      qml.PauliX(wires=2)
      qml.Hadamard(wires=0)
      

for i in range(6):
    @qml.qnode(dev2)
    def circ():
        partial_circuit(i)
        return qml.state()
    state = circ()
    print(f"state at {i}")
    if i != 0:
      drawer = qml.draw(circ)
      print(drawer()) 
    get_state_amplitudes(state)


We can also follow through this circuit mathematically [0, 1, sol]:

$$
|\psi_0\rangle = |0\rangle \otimes |0\rangle \otimes |0\rangle = |000\rangle
$$
Apply $X_{sol}$.
$$
|\psi_1\rangle = |0\rangle \otimes |0\rangle \otimes |1\rangle = |001\rangle
$$
Apply $H_0$ and $H_{sol}$.
$$
|\psi_2\rangle = |+\rangle \otimes |0\rangle \otimes |-\rangle =  \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) \otimes |0\rangle \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = \frac{1}{2}(|000\rangle - |001\rangle + |100\rangle - |101\rangle )
$$
Apply $X_1$
$$
|\psi_3\rangle = |+\rangle \otimes |1\rangle \otimes |-\rangle =  \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) \otimes |1\rangle \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = \frac{1}{2}(|010\rangle - |011\rangle + |110\rangle - |111\rangle )
$$
Apply $Toffoli_{00,sol}$
$$
|\psi_4\rangle = |-\rangle \otimes |1\rangle \otimes |-\rangle =  \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) \otimes |1\rangle \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = \frac{1}{2}(|010\rangle - |011\rangle + |111\rangle - |110\rangle )
$$
Apply $X_1$
$$
|\psi_5\rangle = |-\rangle \otimes |0\rangle \otimes |-\rangle =  \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) \otimes |0\rangle \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = \frac{1}{2}(|000\rangle - |001\rangle + |101\rangle - |100\rangle )
$$
Apply $H_0$ and $H_{sol}$.
$$
|\psi_6\rangle = |1\rangle \otimes |0\rangle \otimes |1\rangle = |101\rangle
$$
Apply $X_{sol}$.
$$
|\psi_7\rangle = |1\rangle \otimes |0\rangle \otimes |0\rangle = |100\rangle
$$

Following the same principles we can map the input to outputs of all combinations.

In [ ]:
numbers = [1, 1]

dev2 = qml.device("default.qubit", wires=[0, 1, 2])

def oracle():
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, 2])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)

@qml.qnode(dev2)
def circuit1():
    # QHACK #
    qml.Hadamard(wires=0)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)
    oracle()
    qml.Hadamard(wires=2)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=0)
    # QHACK #
    return qml.state()

@qml.qnode(dev2)
def circuit2():
    # QHACK #
    qml.Hadamard(wires=1)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)
    oracle()
    qml.Hadamard(wires=2)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=1)
    # QHACK #
    return qml.state()
      
for numbers in [[1,1], [1, 0], [0, 1], [0, 0]]:
  for c in ["c1", "c2"]:
    if c == "c1":
      state = circuit1()
      st = get_state_amplitudes(state)
      print(numbers, c, "bit1=", st[1])
    else:
      state = circuit2()
      st = get_state_amplitudes(state)
      print(numbers, c, "bit2=", st[2])
      

 | Door | Numbers | Circuit | Output | bit1 | bit2 |
| --- | --- | --- | --- | --- | --- |
| 0 | [1, 1] | C1 | 100 | 1 |   | 
| 0 | [1, 1] | C2 | 010 |   | 1 |
| 1 | [1, 0] | C1 | 000 | 0 |   |
| 1 | [1, 0] | C2 | 010 |   | 1 |
| 2 | [0, 1] | C1 | 100 | 1 |   |
| 2 | [0, 1] | C2 | 000 |   | 0 |
| 3 | [0, 0] | C1 | 000 | 0 |   |
| 3 | [0, 0] | C2 | 000 |   | 0 |

Now we can see that the output states clearly return the initial numbers that are used to configure the oracle and with the conditions previously specified, we can get the correct door:

```
bit1 = sol1[0]
bit2 = sol2[1]

if bit1 == 0 and bit2 == 0:
  return 3
elif bit1 == 1 and bit2 == 1:
  return 0
elif bit1 == 1 and bit2 == 0:
  return 2
else:
  return 1
```

The reason behind this behaviour is in the Toffoli gate. It is easy to see that for [0, 0], i.e. door 3, qubits 0 and 1 are never in state $|1\rangle$ at the same time so the outputs are $|000\rangle$ for both circuits. This is also the case for circuit1 when the numbers are [1, 0] and for circuit2 when the numbers are [0, 1]. However, we do have both qubits in state $|1\rangle$ with circuit2 and numbers[1, 0] and for circuit1 and numbers [0, 1]. Lastly, for numbers [1, 1], none of the circuits produces the state $|000\rangle$ since both qubits are in a superposition of state $|1\rangle$ when they reach the Toffoli gate. This creates 4 possible combinations for 4 possible doors. Once one can understand these combinations, it is straightforward to guess the door.